In [25]:
import pandas as pd
import os
import numpy as np

In [26]:
dx = pd.read_csv('../data/mct_combined_v3.csv')
set(dx['question_category'])

{'core counseling attributes',
 'counseling',
 'counseling skills and interventions',
 'intake, assessment, and diagnosis',
 nan,
 'professional practice and ethics',
 'treatment planning'}

In [27]:
def load_dataframe(file, ground_df):
    df = pd.read_csv(file)
    df = df[~pd.isna(df['Question'])]
    question_categories = []
    for row in df.iterrows():
        try:
            question_categories.append(ground_df[ground_df['Question'] == row[1]['Question']].iloc[0]['question_category'])
        except:
            question_categories.append('NA')
    df['question_category'] = question_categories
    df = df[df['question_category'] != "NA"]
    print(df.shape[0])
    return df

def get_avgs_category(df, category):
    dfx = df[df['question_category'] == category]
    try:
        more_ratio = dfx[dfx['case_label'].str.contains('more comprehensive')].shape[0] / dfx.shape[0]
    except:
        more_ratio = -1
    return {'cosine_similarity': np.average(dfx['cosine_similarity']), 'bert_score': np.average(dfx['bert_score']), 'length_difference': np.average(dfx['length_difference']), 'length_prediction': np.average(dfx['length_prediction']), 'more_comprehensive': more_ratio}

In [32]:
generalist_models = ['llama-2-7b-chat-hf', 'Llama-2-70b-hf', 'Llama-2-7b-hf', 'Llama-2-13b-hf', 'Llama-2-70b-hf', 'Llama-2-70b-hf', 'Meta-Llama-3-8B-Instruct', 'Meta-Llama-3-8B-Instruct', 'Meta-Llama-3-70B-Instruct', 'Llama-2-7b-hf', 'Llama-2-13b-chat-hf']
medical_models = ['BioMedGPT-LM-7B', 'med42-70b', 'Asclepius-7B', 'medalpaca-13b', 'meditron-70b', 'ClinicalCamel-70B', 'Llama3-Med42-8B', 'Asclepius-Llama3-8B', 'Llama3-Med42-70B', 'meditron-7b', 'MentaLLaMA-chat-13B']
generalist_models_l = ['Llama-2-70b-hf', 'Llama-2-70b-hf', 'Meta-Llama-3-8B-Instruct', 'Meta-Llama-3-70B-Instruct', 'Llama-2-13b-chat-hf']
medical_models_l = ['meditron-70b', 'ClinicalCamel-70B', 'Llama3-Med42-8B', 'Llama3-Med42-70B', 'MentaLLaMA-chat-13B']

In [33]:
categories = ['intake, assessment, and diagnosis', 'treatment planning', 'professional practice and ethics', 'counseling skills and interventions', 'core counseling attributes']

In [34]:
model_dict = dict([])
for file in os.listdir('../results/outputs_reasoning/'):
    if 'ipynb' in file:
        continue
    df = load_dataframe(os.path.join('../results/outputs_reasoning/', file), dx)
    model_name = file.replace('.csv', '')
    print(model_name)
    model_dict[model_name] = []
    try:
        for category in categories:
            model_dict[model_name].append(get_avgs_category(df, category))
    except:
        print("Need to redo")

1610
Llama-2-13b-hf
Need to redo
1610
med42-70b
1610
Llama-2-70b-chat-hf
1610
llama-2-7b-chat-hf
1610
Llama3-Med42-8B
1610
BioMedGPT-LM-7B
1610
Llama-2-7b-hf
1610
Llama-2-13b-chat-hf
1610
Llama3-Med42-70B
1610
Meta-Llama-3-70B-Instruct
1610
meditron-7b
1610
MentaLLaMA-chat-13B
1610
medalpaca-7b
1610
ClinicalCamel-70B
1610
Llama-2-70b-hf
1610
medalpaca-13b


In [45]:
for model in medical_models_l:
    print('#####################################')
    try:
        print(model)
        for idx, cats in enumerate(model_dict[model]):
            print(categories[idx])
            print(cats)
    except:
        continue

#####################################
meditron-70b
#####################################
ClinicalCamel-70B
intake, assessment, and diagnosis
{'cosine_similarity': 0.28132107056116934, 'bert_score': 0.9956709447103146, 'length_difference': -129.9042207792208, 'length_prediction': 24.43831168831169, 'more_comprehensive': 0.06926406926406926}
treatment planning
{'cosine_similarity': 0.25357181585474514, 'bert_score': 0.9999999347634203, 'length_difference': -132.65846456692913, 'length_prediction': 20.561023622047244, 'more_comprehensive': 0.06299212598425197}
professional practice and ethics
{'cosine_similarity': 0.20087749107430378, 'bert_score': 0.9999999235837888, 'length_difference': -142.4981684981685, 'length_prediction': 17.57234432234432, 'more_comprehensive': 0.054945054945054944}
counseling skills and interventions
{'cosine_similarity': 0.30796388455883444, 'bert_score': 0.9999999456868899, 'length_difference': -139.9250418760469, 'length_prediction': 28.048157453936348, 'more_